# Part 1 - Plot GDP Data

In [1]:
import pandas as pd
import re
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from io import StringIO

GDP_COL_NAME = "GDP (US$ million)"

def retrieve_data_for_region(region_name, region_url, gdp_col_match=r'GDP.*', two_rank_cols=False, special_europe=False):
    req = Request(region_url, headers={'User-Agent': 'Mozilla/5.0'})
    page = urlopen(req).read()
    soup = BeautifulSoup(page, "html.parser")
    tables = soup.find_all("table")    

    # Find the GDP table
    found_GDP_table = None
    for table in tables:                
        headers = table.find_all('th')        
        # Find the IMF GDP data for this table

        for header in headers:
            # print(f"  Header: {header.get_text()}")
            # print(f"    Matching against: {gdp_col_match}")
            if re.search(gdp_col_match, header.get_text(), re.IGNORECASE):                
                found_GDP_table = table
                break
        
        if found_GDP_table:
            break

    if not found_GDP_table:
        raise ValueError(f"Could not find the GDP table on the page for {region_name}.")

    # Get the dataframe from the table
    gdp_df = pd.read_html(StringIO(str(found_GDP_table)))[0]

    # If two rank columns, just delete the second column to normalize
    if two_rank_cols:
        gdp_df = gdp_df.drop(gdp_df.columns[1], axis=1)

    if special_europe or region_name == "Oceania":
        # There is no rank column, insert a dummy one with 0s (doesn't matter)        
        gdp_df.insert(0, 'Rank', 0)        
    
    if special_europe:
        # Now the column we care about will be the 3rd real col (skip 2030), so delete the second
        gdp_df = gdp_df.drop(gdp_df.columns[2], axis=1)                
    
    # Filter out rows where the first column is not an integer (no rank)
    gdp_df = gdp_df[gdp_df.iloc[:, 0].apply(lambda x: str(x).strip().isdigit())]                    

    # The column we care about will now be 1 and 2
    gdp_df = gdp_df.iloc[:, [1, 2]]

    gdp_df.columns = ['Country', GDP_COL_NAME]    

    # Force numeric conversion on the GDP column, if GDP column is a string
    if gdp_df[GDP_COL_NAME].dtype == object:
        gdp_df[GDP_COL_NAME] = pd.to_numeric(gdp_df[GDP_COL_NAME].str.replace(',', ''), errors='coerce')

    # Filter out NaNs (small territories, or folded into other countries)
    gdp_df = gdp_df.dropna(subset=[GDP_COL_NAME])        

    # Add the territory name as a column for each row
    gdp_df = gdp_df.assign(Region=region_name)

    return gdp_df

def retrieve_gdp_data_for_regions():
    # The regions of the globe from the main wiki page (commonwealth is not really a region so we can omit)
    
    ap_df = retrieve_data_for_region("Asia-Pacific", "https://en.wikipedia.org/wiki/List_of_countries_in_Asia-Pacific_by_GDP_(nominal)", r"GDP.*")    
    al_df = retrieve_data_for_region("Arab League", "https://en.wikipedia.org/wiki/List_of_Arab_League_countries_by_GDP_(nominal)", r"Total GDP")
    # Normalize Arab League to $millions instead of $billions as on the page
    al_df = al_df.assign(**{GDP_COL_NAME: al_df[GDP_COL_NAME] * 1000})

    afr_df = retrieve_data_for_region("Africa", "https://en.wikipedia.org/wiki/List_of_African_countries_by_GDP_(nominal)", r"Nominal GDP.*")
    lam_df = retrieve_data_for_region("Latin America", "https://en.wikipedia.org/wiki/List_of_Latin_American_and_Caribbean_countries_by_GDP_(nominal)", r"GDP \(nominal\) \(millions.*")
    oc_df = retrieve_data_for_region("Oceania", "https://en.wikipedia.org/wiki/List_of_Oceanian_countries_by_GDP", r"GDP")
    
    na_df = retrieve_data_for_region("North America", "https://en.wikipedia.org/wiki/List_of_North_American_countries_by_GDP_(nominal)", r"GDP.*", two_rank_cols=True)
    eu_df = retrieve_data_for_region("Europe", "https://en.wikipedia.org/wiki/List_of_sovereign_states_in_Europe_by_GDP_(nominal)", r"2030 \(project", special_europe=True)    

    # Combine the dataframes
    df = pd.concat([ap_df, al_df, afr_df, lam_df, oc_df, na_df, eu_df], ignore_index=True)
    
    return df

# Convenient test for debugging
gdp_df = retrieve_gdp_data_for_regions()

print(gdp_df)

             Country  GDP (US$ million)        Region
0              China         17700899.0  Asia-Pacific
1              Japan          4230862.0  Asia-Pacific
2              India          3732224.0  Asia-Pacific
3        South Korea          1709232.0  Asia-Pacific
4          Australia          1687713.0  Asia-Pacific
..               ...                ...           ...
222  North Macedonia            17885.0        Europe
223           Kosovo            11274.0        Europe
224       Montenegro             8562.0        Europe
225          Andorra             4035.0        Europe
226       San Marino             2047.0        Europe

[227 rows x 3 columns]


In [ ]:
# Now we can generate a plotly figure
import plotly.express as px

# create a stacked bar chart for GDP, by region
fig = px.bar(gdp_df, x='Region', 
             y=GDP_COL_NAME, color='Country', 
             title='GDP by Country and Region, in "trillions" (millions of millions)', 
             barmode='relative')

# For copy to github pages
fig.write_html("stacked_bar.html")

fig.show()

# Part 2 - Generate sankey plot for intracranial region breakdowns

In [131]:
import plotly.graph_objects as go
import urllib, json
import numpy as np
import pandas as pd
import plotly.express as px

# Load the lookup table with the levels
url = "https://raw.githubusercontent.com/bcaffo/MRIcloudT1volumetrics/master/inst/extdata/multilevel_lookup_table.txt"
multilevel_lookup = pd.read_csv(url, sep = "\t").drop(['Level5'], axis = 1)
multilevel_lookup = multilevel_lookup.rename(columns = {
    "modify"   : "roi",
    "modify.1" : "level4",
    "modify.2" : "level3",
    "modify.3" : "level2",
    "modify.4" : "level1"})
multilevel_lookup = multilevel_lookup[['roi', 'level4', 'level3', 'level2', 'level1']]

## Load the subject data, we only care about type 1
id = 127
df = pd.read_csv("https://raw.githubusercontent.com/smart-stats/ds4bio_book/main/book/assetts/kirby21AllLevels.csv")
df = df.loc[(df.type == 1) & (df.id == id)]
df = df[['roi', 'volume']]

## Merge the subject data with the multilevel data
df = pd.merge(df, multilevel_lookup, on = "roi")
df = df.assign(icv = "ICV")

# Fix CSF row - fill the weird xxx levels with 'CSF' to make it consistent with other rows
csf_mask = df['roi'] == 'CSF'
df.loc[csf_mask, 'level4'] = 'CSF'
df.loc[csf_mask, 'level3'] = 'CSF'
df.loc[csf_mask, 'level2'] = 'CSF' 
df.loc[csf_mask, 'level1'] = 'CSF'

# Calculate total ICV volume for the whole brain for normalization
total_icv_volume = df['volume'].sum()

# Create hierarchical structure with aggregated volumes at each level
# Let's leave off ROI, clutters the diagram too much
levels = ['icv', 'level1', 'level2', 'level3', 'level4']  

# Calculate node volumes for each level, just sum all the values that match that level name
node_volumes = {}
for level in levels:
    if level == 'icv':
        node_volumes[level] = {'ICV': total_icv_volume}
    else:
        level_df = df.groupby([level]).agg({'volume': 'sum'}).reset_index()
        node_volumes[level] = dict(zip(level_df[level], level_df['volume']))

# Create all nodes with their positions (which level)
all_nodes = []
node_positions = {}  # (level_index, node_name) -> node_index
node_volumes_flat = []  # just volume for each unique node
node_levels = []  # level index for each node
node_labels = []  # display label for each node

for level_idx, level in enumerate(levels):
    nodes_in_level = list(node_volumes[level].keys())
    for node_name in nodes_in_level:        
        node_index = len(all_nodes)
        all_nodes.append((level_idx, node_name))
        node_positions[(level_idx, node_name)] = node_index
        node_volumes_flat.append(node_volumes[level][node_name])
        node_levels.append(level_idx)
        node_labels.append(node_name)

# Create links between levels
sources = []
targets = []
values = []
link_colors = []

# Reuse a light palette with good contrast
color_palette = px.colors.qualitative.Set3

unique_regions = list(set([node[1] for node in all_nodes]))
region_colors = {}

# Map each unique region to a color - this will just rotate rather than keeping the same color for each "type" of region
# since it's easier to see with contrast
for i, region in enumerate(unique_regions):
    region_colors[region] = color_palette[i % len(color_palette)]

# ICV will always be gray for readability
region_colors['ICV'] = 'gray'

# Create links by tracing each unique path through the hierarchy with collapsing duplicates
for _, row in df.iterrows():
    # Create each path
    path = []
    for level in levels:
        path.append(row[level])
    
    # Collapse consecutive duplicates in the path (so we don't see "CSF" repeated each level for instance, it stays Level 1)
    # This is going to look a bit weird in some ways (only part of the vertical box has lines coming out of it) but reduces the noise
    collapsed_path = []
    for i, node in enumerate(path):
        if i == 0 or node != collapsed_path[-1]:
            collapsed_path.append(node)
    
    # Create links for the proper path
    for i in range(len(collapsed_path) - 1):
        source_node = collapsed_path[i]
        target_node = collapsed_path[i + 1]
        
        # Find the level indices for source and target nodes
        source_level_idx = None
        target_level_idx = None
        
        # Find which level each node belongs to by checking node_volumes
        for level_idx, level in enumerate(levels):
            if source_node in node_volumes[level]:
                source_level_idx = level_idx
            if target_node in node_volumes[level]:
                target_level_idx = level_idx
        
        # If the link has a place to go to and from, then add it!
        if source_level_idx is not None and target_level_idx is not None:
            source_idx = node_positions[(source_level_idx, source_node)]
            target_idx = node_positions[(target_level_idx, target_node)]
            
            sources.append(source_idx)
            targets.append(target_idx)
            values.append(row['volume'])
            
            # Make the hover slightly transparent
            link_colors.append(region_colors[source_node].replace('rgb', 'rgba').replace(')', ', 0.8)'))

# Get the node colors back from the map
node_colors = [region_colors[node[1]] for node in all_nodes]

# Create Sankey diagram with the node data
fig = go.Figure(go.Sankey(
    node=dict(
        pad=30,
        thickness=20,
        line=dict(color="black", width=1.5),
        label=node_labels,
        color=node_colors,        
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color=link_colors,        
    )
))

# Add the title, fix the layout
fig.update_layout(
    title=dict(
        text="Brain Region ICV Contribution Flow",
        x=0.5,
        xanchor='center',
        y=0.98,
        yanchor='top',
        font=dict(size=16, color='black')
    ),
    font=dict(size=12),
    width=2000,
    height=2200,
    plot_bgcolor='white',
    paper_bgcolor='white'
)

# Put the column names
level_names = ['ICV', 'Level 1', 'Level 2', 'Level 3', 'Level 4']
for i, level_name in enumerate(level_names):
    fig.add_annotation(
        x=i/4, # Evenly space
        y=1.03,
        xref="paper", 
        yanchor="top",
        text=level_name,        
        showarrow=False,
        font=dict(size=16, color="black"),        
    )

# Better margin than defaults
fig.update_layout(margin=dict(t=150, l=50, r=50, b=50))

# For copy to github pages
fig.write_html("sankey.html")

fig.show()

# Diagram Links
* Stacked Bar Chart: https://mberenbach.github.io/ds4ph-bme-assignment5/stacked_bar.html
* Sankey ICV Chart: https://mberenbach.github.io/ds4ph-bme-assignment5/sankey.html